In [1]:
import data_loader
import numpy as np
import keras
import pickle
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils
from keras.optimizers import SGD

Using TensorFlow backend.


### Load the original data

In [2]:
img_dir_path = 'origin/images'
label_file_path = 'origin/data/legend.csv'
valid_rate = 0.1

train_file_paths, \
train_labels, \
valid_file_paths, \
valid_labels, \
test_file_paths, \
test_labels, \
label_dict = data_loader.load_dataset(img_dir_path, label_file_path, valid_rate)

print('train_images size is: ', len(train_file_paths))
print('valid_images size is: ', len(valid_file_paths))
print('test_images size is: ', len(test_file_paths))
print('label_dict size is: ', len(label_dict))
print(label_dict.keys())

train_images size is:  11086
valid_images size is:  1226
test_images size is:  1378
label_dict size is:  8
dict_keys(['surprise', 'sadness', 'contempt', 'fear', 'anger', 'neutral', 'disgust', 'happiness'])


### Load the extracted features by CNN

In [3]:
with open("extract/x_train_norm", "rb") as f:
    x_train = np.array(pickle.load(f))

with open("extract/x_valid_norm", "rb") as f:
    x_valid = np.array(pickle.load(f))

with open("extract/x_test_norm", "rb") as f:
    x_test = np.array(pickle.load(f))

y_train = np_utils.to_categorical(train_labels, 8)
y_valid = np_utils.to_categorical(valid_labels, 8)
y_test = np_utils.to_categorical(test_labels, 8)

print("x_train is", x_train.shape)
print("x_valid is", x_valid.shape)
print("x_test is", x_test.shape)

print("y_train is", y_train.shape)
print("y_valid is", y_valid.shape)
print("y_test is", y_test.shape)

x_train is (11086, 512)
x_valid is (1226, 512)
x_test is (1378, 512)
y_train is (11086, 8)
y_valid is (1226, 8)
y_test is (1378, 8)


In [21]:
from keras import backend as K

def loss_function(y_true, y_pred):
    print(K.int_shape(y_true))
    print(K.int_shape(y_pred))
    return K.mean((y_true - y_pred)**2)

In [6]:
model = Sequential()
model.add(Dense(units=256, activation='relu', input_dim=512))
model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=8, activation='softmax'))

optimizer = SGD(lr=1e-3, momentum=0.8)
model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ['accuracy'])

model.fit(x_train, y_train, epochs=10, steps_per_epoch=1000, validation_steps=100,validation_data=(x_test, y_test))
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)

Train on 11086 samples, validate on 1378 samples
Epoch 1/10
  80/1000 [=>............................] - ETA: 1:00 - loss: 0.3732 - acc: 0.8750

KeyboardInterrupt: 